In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/fashion_store_sales.csv')

In [3]:
df.shape

(2253, 29)

In [8]:
df.describe()

,item_id,sale_id,product_id,quantity,original_price,unit_price,discount_applied,discounted,item_total,total_amount,catalog_price,cost_price,customer_id
count,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2028.000000,2253.000000,2253.000000,2253.000000
mean,1696.373724,613.966711,254.117621,2.980470,49.321185,48.141074,1.180129,0.098535,143.913298,372.860449,49.321185,27.153964,503.597870
std,964.499872,383.218568,145.214862,1.420658,13.077055,13.373302,3.996179,0.298103,82.153406,152.768703,13.077055,8.639542,290.481602
min,2.000000,2.000000,1.000000,1.000000,13.510000,11.680000,0.000000,0.000000,13.510000,45.500000,13.510000,6.850000,1.000000
25%,877.000000,288.000000,127.000000,2.000000,39.340000,38.490000,0.000000,0.000000,74.700000,257.310000,39.340000,20.730000,235.000000
50%,1679.000000,559.000000,253.000000,3.000000,49.760000,47.730000,0.000000,0.000000,130.500000,365.085000,49.760000,26.380000,505.000000
75%,2515.000000,942.000000,383.000000,4.000000,58.340000,57.350000,0.000000,0.000000,198.750000,472.800000,58.340000,32.740000,759.000000
max,3367.000000,1352.000000,500.000000,5.000000,85.900000,85.900000,24.510000,1.000000,403.800000,859.380000,85.900000,53.760000,999.000000


In [4]:
df.columns

Index(['sale_date', 'item_id', 'sale_id', 'product_id', 'quantity',
       'original_price', 'unit_price', 'discount_applied', 'discount_percent',
       'discounted', 'item_total', 'channel', 'channel_campaigns',
       'total_amount', 'product_name', 'category', 'brand', 'color', 'size',
       'catalog_price', 'cost_price', 'customer_id', 'gender', 'age_range',
       'signup_date', 'first_name', 'last_name', 'email', 'country'],
      dtype='str')

In [5]:
df.sale_id.unique().shape

(905,)

In [12]:
df.duplicated().sum()

np.int64(0)

In [6]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 2253 entries, 0 to 2252
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sale_date          2253 non-null   str    
 1   item_id            2253 non-null   int64  
 2   sale_id            2253 non-null   int64  
 3   product_id         2253 non-null   int64  
 4   quantity           2253 non-null   int64  
 5   original_price     2253 non-null   float64
 6   unit_price         2253 non-null   float64
 7   discount_applied   2253 non-null   float64
 8   discount_percent   2253 non-null   str    
 9   discounted         2253 non-null   int64  
 10  item_total         2253 non-null   float64
 11  channel            2253 non-null   str    
 12  channel_campaigns  2253 non-null   str    
 13  total_amount       2028 non-null   float64
 14  product_name       2253 non-null   str    
 15  category           2253 non-null   str    
 16  brand              2253 non-null   

In [7]:
# Get columns with missing values
cols_with_na = df.columns[df.isna().any() ]
print("Colonne avec des valeurs manquantes :", cols_with_na.tolist())
print(df[cols_with_na].isna().sum() / len(df) * 100)  # Pourcentage de valeurs manquantes par colonne

Colonne avec des valeurs manquantes : ['total_amount', 'first_name', 'last_name', 'email']
total_amount    9.986684
first_name      5.148691
last_name       2.840657
email           9.942299
dtype: float64


In [20]:
for col in ['channel','category','brand', 'gender', 'age_range', 'country', 'size'] : 
  print(df[col].value_counts(), end='\n\n\n')
  

channel
E-commerce    1170
App Mobile    1083
Name: count, dtype: int64


category
T-Shirts     492
Dresses      482
Shoes        472
Sleepwear    455
Pants        352
Name: count, dtype: int64


brand
Tiva    2253
Name: count, dtype: int64


gender
Female    2253
Name: count, dtype: int64


age_range
36-45    477
26-35    463
16-25    455
46-55    433
56-65    425
Name: count, dtype: int64


country
Germany        537
France         498
Italy          415
Netherlands    326
Spain          276
Portugal       201
Name: count, dtype: int64


size
XS    493
S     358
XL    354
L     350
M     322
36    109
38     93
35     92
40     82
Name: count, dtype: int64




In [21]:
df.sale_date = pd.to_datetime(df.sale_date)

In [24]:
df.sale_date.agg(['min', 'max'])

min   2025-04-04
max   2025-06-17
Name: sale_date, dtype: datetime64[us]

```text
Le dataset regroupe plusieurs entités métier dans une seule table, ce qui induit des redondances importantes.
Par exemple pour les produits, les informations sont répétées pour chaque ligne de vente contenant le même produit. Il en est de même pour les informations du client.
```

# Modélisation et normalisation

## Champs exclus du schema normalise

Le dataset source contient 29 colonnes. Le schema normalise n'en retient que 21. Les colonnes suivantes ont ete volontairement exclues :

| Colonne | Raison de l'exclusion |
| --- | --- |
| `original_price` | Identique a `catalog_price` (redondance directe). Conserver les deux violerait la 2NF : deux attributs portent la meme information pour une meme cle. |
| `unit_price` | Calculable a partir de `catalog_price - discount_applied`. Stocker une valeur derivee viole la 3NF (dependance transitive : `item_id` -> `catalog_price, discount_applied` -> `unit_price`). |
| `discount_percent` | Calculable a partir de `discount_applied / catalog_price * 100`. Meme violation de 3NF que `unit_price`. De plus, cette colonne est de type `str` dans le CSV alors qu'elle devrait etre numerique, ce qui confirme un probleme de qualite. |
| `discounted` | Booleen derivable de `discount_applied > 0`. Aucune information nouvelle par rapport a `discount_applied`. |
| `item_total` | Calculable via `quantity * unit_price`. Dependance transitive sur des attributs deja presents dans `sale_product` et `product`. |
| `total_amount` | Somme des `item_total` par `sale_id`. C'est un aggregat, pas un attribut atomique. Le stocker violerait la 3NF et introduirait des risques d'incoherence si un item est modifie sans recalculer le total. De plus, cette colonne contient ~10% de valeurs manquantes, ce qui renforce le choix de la recalculer a la demande. |

En resume : ne sont conserves que les attributs atomiques, non derivables, qui dependent directement de la cle primaire de leur table. Les valeurs calculees seront reconstituees par des requetes SQL au moment de l'exploitation.

## Entites et attributs

- Client :  
  - customer_id (identifiant)
  - first_name
  - last_name
  - email
  - country
  - signup_date
  - gender
  - age_range

- Produit : 
  - product_id (identifiant)
  - product_name
  - brand
  - category
  - cost_price
  - color
  - size
  - catalog_price

- Sale
  - sale_id (identifiant)
  - sale_date
  - channel
  - channel_campaigns
  - customer_id

- Sale_Product
  - item_id (identifiant)
  - sale_id
  - product_id
  - quantity
  - discount_applied


## Schema de la bd
<img src='schema_bd.png'/>